## Importing Libraries

### Description
This section of the code is responsible for importing the necessary libraries that will be used in the program.


In [7]:
import os
from IPython import display
display.clear_output()
from IPython.display import display, Image

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Setup complete ✅ (20 CPUs, 15.4 GB RAM, 111.2/1006.9 GB disk)


In [8]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Uf5F9mzBsL27ozIYiWvW")
project = rf.workspace("mangguai").project("obstacle-detection-pgo7d")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Obstacle-Detection-3 in yolov8:: 100%|██████████| 5237/5237 [00:00<00:00, 27400.52it/s]


In [9]:
import os

# Define the classes and their corresponding indices
class_names = ['nojaywalking_sign','mandatorystop_sign','curveright_sign','zebracrossing_sign','noentry_sign','pedestriancrossing_sign','curveleft_sign']
class_indices = {name: index for index, name in enumerate(class_names)}

# Image dimensions
img_width, img_height = 640, 640

# Path to the folder containing the annotation files
folder_path = 'Obstacle-Detection-3/train/labels'
output_folder_path = 'Obstacle-Detection-3/train/modified_labels'  # New output folder

os.makedirs(output_folder_path, exist_ok=True)  # Ensure the output directory exists

# Function to normalize coordinates
def normalize_coordinates(coord, max_value):
    return float(coord) / max_value

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        output_file_path = os.path.join(output_folder_path, filename)  # Output file path
        with open(file_path, 'r') as file:
            lines = file.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split(' ')
            if len(parts) == 10:
                # Extract label and coordinates
                label = parts[-2]
                coords = parts[:8]

                # Get the class index
                class_index = class_indices.get(label, -1)
                if class_index != -1:
                    # Normalize coordinates
                    normalized_coords = [normalize_coordinates(coord, img_width if i % 2 == 0 else img_height) for i in range(8)]

                    # Convert to the desired format and add to new lines
                    new_line = f"{class_index} " + " ".join(map(str, normalized_coords))
                    new_lines.append(new_line)

        # Write the converted lines to a new file in the output directory
        with open(output_file_path, 'w') as file:
            file.write('\n'.join(new_lines))


In [14]:
import subprocess

# Command to run
command = "yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Obstacle-Detection-3/data.yaml epochs=20 imgsz=640 batch=8"

# Execute the command
subprocess.run(command, shell=True)

New https://pypi.org/project/ultralytics/8.2.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/mangguai/capstone/code/Obstacle-Detection-3/data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, lin

CompletedProcess(args='yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Obstacle-Detection-3/data.yaml epochs=20 imgsz=640 batch=8', returncode=0)

In [57]:
import cv2
import torch
import numpy as np

# Function to load the model
def load_model(model_path):
    model = torch.load(model_path, map_location=torch.device('cpu'))['model']  # Load to CPU
    model.eval()
    return model

# Preprocess the frame
def preprocess_frame(frame, img_size=640):
    frame = cv2.resize(frame, (img_size, img_size))  # Resize to model's input size
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   # Convert BGR to RGB
    frame = np.transpose(frame, (2, 0, 1))           # Reshape from HWC to CHW format
    frame = np.expand_dims(frame, axis=0)            # Add batch dimension
    frame = frame / 255.0                            # Normalize to [0, 1]
    frame = torch.from_numpy(frame).float()          # Convert to float tensor
    return frame

# Function to draw detections on frame
def draw_boxes(detections, frame):
    for detection in detections:
        xmin, ymin, xmax, ymax, conf, cls = int(detection[0]), int(detection[1]), int(detection[2]), int(detection[3]), detection[4], int(detection[5])
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        label = f'Class {cls}: {conf:.2f}'
        cv2.putText(frame, label, (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Load the model
model = load_model('/home/mangguai/capstone/code/runs/detect/train/weights/best.pt')
model = YOLO('yolov8s-world.pt')
# Open a video
cap = cv2.VideoCapture('../data/singapore_downtown.mp4')

# Process video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    input_tensor = preprocess_frame(frame)
    
    # Get predictions
    with torch.no_grad():
        predictions = model(input_tensor)
        predictions = predictions[0].cpu().numpy()  # Convert predictions to numpy array

    # Filter predictions with confidence > threshold
    conf_thresh = 0.5
    predictions = predictions[predictions[:, 4] > conf_thresh]

    # Draw detections on the frame
    draw_boxes(predictions, frame)

    # Display the frame
    cv2.imshow('Video Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [99]:
import cv2
import torch
import numpy as np

# Function to load the model
def load_model(model_path):
    model = torch.load(model_path, map_location=torch.device('cpu'))['model'].eval()  # Load to CPU
    return model

# Preprocess the frame
def preprocess_frame(frame, img_size=640):
    # Resize and normalize the frame
    frame = cv2.resize(frame, (img_size, img_size))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.transpose(frame, (2, 0, 1)) / 255.0  # Reshape and normalize
    frame = torch.from_numpy(frame).float().unsqueeze(0)  # Add batch dimension
    return frame

# Function to draw detections on the frame
def draw_boxes(detections, frame, img_size=640):
    scale_w, scale_h = frame.shape[1] / img_size, frame.shape[0] / img_size
    for detection in detections:
        xmin, ymin, xmax, ymax, conf, cls_conf, cls_pred = detection
        xmin, xmax = int(xmin * scale_w), int(xmax * scale_w)
        ymin, ymax = int(ymin * scale_h), int(ymax * scale_h)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        label = f'{cls_pred} {conf:.2f}'
        cv2.putText(frame, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Load the model
model_path = '../../code/runs/detect/train/weights/best.pt'  # Update with the path to your model file
model = load_model(model_path)

# Open a video
video_path = '../data/singapore_downtown.mp4'  # Update with the path to your video file
cap = cv2.VideoCapture(video_path)  # Initialize the VideoCapture object

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for the model
    input_tensor = preprocess_frame(frame)

    # Get predictions from the model
    with torch.no_grad():
        predictions = model(input_tensor)
        predictions = predictions[0].xyxy[0].numpy()  # Output format is [x_min, y_min, x_max, y_max, confidence, class_confidence, class_pred]

    # Filter out predictions with low confidence
    predictions = predictions[predictions[:, 4] > 0.3]  # Confidence threshold

    # Draw boxes on the frame
    draw_boxes(predictions, frame)

    # Display the frame
    cv2.imshow('Object Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [101]:
pwd

'/home/mangguai/capstone/code/Obstacle-Detection-3'

In [102]:
#before running this command, make sure you are in the correct directory, check with pwd
!yolo task=detect mode=predict model=best.pt conf=0.25 source=singapore_downtown.mp4

Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 11128293 parameters, 0 gradients, 28.5 GFLOPs

video 1/1 (1/24575) /home/mangguai/capstone/code/Obstacle-Detection-3/singapore_downtown.mp4: 384x640 (no detections), 90.7ms
video 1/1 (2/24575) /home/mangguai/capstone/code/Obstacle-Detection-3/singapore_downtown.mp4: 384x640 (no detections), 5.5ms
video 1/1 (3/24575) /home/mangguai/capstone/code/Obstacle-Detection-3/singapore_downtown.mp4: 384x640 (no detections), 6.0ms
video 1/1 (4/24575) /home/mangguai/capstone/code/Obstacle-Detection-3/singapore_downtown.mp4: 384x640 (no detections), 6.0ms
video 1/1 (5/24575) /home/mangguai/capstone/code/Obstacle-Detection-3/singapore_downtown.mp4: 384x640 (no detections), 6.5ms
video 1/1 (6/24575) /home/mangguai/capstone/code/Obstacle-Detection-3/singapore_downtown.mp4: 384x640 (no detections), 5.0ms
video 1/1 (7/24575) /home/mangguai/capstone/code/Obstacle-Det